In [26]:
import pandas as pd
import os
import openpyxl
import docx

In [27]:
THRESHOLD=500
doc = docx.Document() 

In [28]:
def drivers(input_data):
    driver_dict = {'Net Revenue': {'label': 'revenue', 'var': 0, 'var_str':'', 'value_status': 0,'comment_str' : ''},
                    'COGS': {'label': 'Cost of Goods Sold', 'var': 0, 'var_str':'', 'value_status': 0,'comment_str' : ''},
                    'OPEX': {'label': 'Operating Expenses', 'var': 0, 'var_str':'', 'value_status': 0,'comment_str' : ''},
                    }

    for driver in driver_dict.keys():
        
        driver_dict[driver]['var'] = input_data.loc[driver,'var'].astype(int)
        percen = round(((input_data.loc[driver,'Current Year'].astype(int) - input_data.loc[driver,'Prior Year'].astype(int))/input_data.loc[driver,'Prior Year'].astype(int))*100)
        # if abs(driver_dict[driver]['var']) < 1e3:
        #     driver_dict[driver]['var_str'] = f"${abs(driver_dict[driver]['var']):.0f}K"
        if abs(driver_dict[driver]['var']) < 1e6:
            driver_dict[driver]['var_str']  = f"${(abs(driver_dict[driver]['var'])+1e-3)/1e3:.0f}m"
        else:
            driver_dict[driver]['var_str']  = f"${abs(driver_dict[driver]['var'])/1e6:.1f}bn"
        
        if driver_dict[driver]['var'] <= THRESHOLD*-1:
            driver_dict[driver]['value_status'] = -1 if driver == 'Net Revenue' else 1
            driver_dict[driver]['comment_str'] = f"{driver_dict[driver]['var_str']} ({percen}%) lower {driver_dict[driver]['label']}"
        elif driver_dict[driver]['var'] >= THRESHOLD:
            driver_dict[driver]['value_status'] = 1 if driver == 'Net Revenue' else -1
            driver_dict[driver]['comment_str'] = f"{driver_dict[driver]['var_str']}({percen}%) higher {driver_dict[driver]['label']}"
        else:
            driver_dict[driver]['value_status'] = 0
            driver_dict[driver]['comment_str'] = f"{driver_dict[driver]['label']} are flat to prior year"

    driver_df = pd.DataFrame(driver_dict).T
    driver_df['abs_var'] = driver_df['var'].abs()

    pos_drivers = driver_df[driver_df['value_status'] == 1].sort_values(by='abs_var',ascending=False)
    neg_drivers = driver_df[driver_df['value_status'] == -1].sort_values(by='abs_var',ascending=False)
    flat_drivers = driver_df[driver_df['value_status'] == 0]

    driver_comments = {}

    for drivers,driver_name in [[pos_drivers,f"{pos_drivers=}".split("=")[0]],[neg_drivers,f"{neg_drivers=}".split("=")[0]],[flat_drivers,f"{flat_drivers=}".split("=")[0]]]:
        driver_comments = {**driver_comments,driver_name:''}
        cnt = drivers.shape[0]

        if cnt == 1:
            driver_comments[driver_name] = drivers['comment_str'].values[0]
        if cnt > 1:
            for i in range(cnt):
                driver_comments[driver_name] += drivers['comment_str'].values[i]
                
                if i == cnt-2:
                    driver_comments[driver_name] += ' and '
                elif cnt > 1 and i < cnt-1:
                    driver_comments[driver_name] += ', '
    return driver_comments

In [29]:
def commentary(input_data):
    
    driver_comments = drivers(input_data)

    pl = input_data.loc['PBT','Current Year'].astype(int)
    pl_var = input_data.loc['PBT','var'].astype(int)
    plprev = input_data.loc['PBT','Prior Year'].astype(int)
    plp = int(((pl-plprev)/plprev)*100)
    driver_comment = ''

    # if abs(pl_var) < 1e3:
    #     pl_var_str = f"${abs(pl_var):.0f}K"
    if abs(pl_var) < 1e6:
        pl_var_str = f"${(abs(pl_var)+1e-3)/1e3:.0f}m"
    else:
        pl_var_str = f"${abs(pl_var)/1e6:.1f}bn"

    if pl < 0:
        if pl_var <= THRESHOLD*-1:
            pl_status = f"Loss is {pl_var_str} ({plp}%) higher than"
            driver_comment = f"{driver_comments['neg_drivers']}{' partly offset by '+driver_comments['pos_drivers']+'.'  if driver_comments['pos_drivers'] != '' else '.'}{' '+driver_comments['flat_drivers']+'.' if driver_comments['flat_drivers'] != '' else ''}"
            
        elif pl_var >= THRESHOLD:
            pl_status = f"Loss is {pl_var_str} ({plp}%)  lower than"
            driver_comment = f"{driver_comments['pos_drivers']}{' partly offset by '+driver_comments['neg_drivers']+'.'  if driver_comments['neg_drivers'] != '' else '.'}{' '+driver_comments['flat_drivers']+'.' if driver_comments['flat_drivers'] != '' else ''}"
        else:
            pl_status = "Loss is nearly flat to"
            driver_comment = f"{driver_comments['pos_drivers']}{' offset by '+driver_comments['neg_drivers']+'.'  if driver_comments['neg_drivers'] != '' else '.'}{' '+driver_comments['flat_drivers']+'.' if driver_comments['flat_drivers'] != '' else ''}"
    else:
        if pl_var >= THRESHOLD:
            pl_status = f"Profit is {pl_var_str} ({plp}%)  higher than"
            driver_comment = f"{driver_comments['pos_drivers']}{' partly offset by '+driver_comments['neg_drivers']+'.'  if driver_comments['neg_drivers'] != '' else '.'}{' '+driver_comments['flat_drivers']+'.' if driver_comments['flat_drivers'] != '' else ''}"
        elif pl_var <= THRESHOLD*-1:
            pl_status = f"Profit is {pl_var_str}  ({plp}%) lower than"
            driver_comment = f"{driver_comments['neg_drivers']}{' partly offset by '  if driver_comments['pos_drivers'] != '' else '.'}{driver_comments['pos_drivers']}.{' '+driver_comments['flat_drivers']+'.' if driver_comments['flat_drivers'] != '' else ''}"
        else:
            pl_status = "Profit is nearly flat to"
            driver_comment = f"{driver_comments['pos_drivers']}{' offset by '+driver_comments['neg_drivers']+'.'  if driver_comments['neg_drivers'] != '' else '.'}{' '+driver_comments['flat_drivers']+'.' if driver_comments['flat_drivers'] != '' else ''}"       

    return f"""{pl_status} prior year driven by {driver_comment}"""

In [30]:
def roundoff(num):
     num  = round((abs(num)+1e-3)/1e3)
     return num

In [31]:
comments=''
inputs_df = pd.read_excel('Modified_Data_Sets.xlsx',sheet_name=None,header=1,index_col=1)
for df in inputs_df.keys():
                input_df = inputs_df[df]
                input_df.dropna(axis =1,how = 'all',inplace=True)
                input_df.dropna(how = 'all',inplace=True)
                input_df['var'] = input_df['Current Year'] - input_df['Prior Year']
                input_df = input_df.loc[['Net Revenue','COGS','OPEX','PBT'],['Current Year','var', 'Prior Year']]  
                comment = commentary(input_df)
                comments += f"{df} - comment: {comment}\n\n"
                        
doc.add_paragraph(comments)
doc.save('Comments.docx')
print(comments)

Example 1 - comment: Profit is $24m (162%)  higher than prior year driven by $22m(24%) higher revenue partly offset by $1m(-2%) higher Cost of Goods Sold and $1m(-5%) higher Operating Expenses.

Example 2 - comment: Profit is $23m (160%)  higher than prior year driven by $22m(24%) higher revenue partly offset by $1m(-2%) higher Cost of Goods Sold. Operating Expenses are flat to prior year.

Example 3 - comment: Loss is $18m (50%) higher than prior year driven by $18m (-22%) lower revenue. Cost of Goods Sold are flat to prior year and Operating Expenses are flat to prior year.

Example 4 - comment: Loss is nearly flat to prior year driven by $2m (15%) lower Operating Expenses offset by $20m(-19%) higher Cost of Goods Sold and $18m (-22%) lower revenue.

Example 5 - comment: Loss is $34m (-96%)  lower than prior year driven by $2m (15%) lower Operating Expenses partly offset by $54m(-51%) higher Cost of Goods Sold and $18m (-22%) lower revenue.

Example 6 - comment: Profit is $20m (44%) 

In [32]:
def roundoff(num):
     num  = round((abs(num)+1e-3)/1e3)
     return num

In [33]:
inputs_df = pd.read_excel('Modified_Data_Sets.xlsx',sheet_name=None,header=1,index_col=1)
for df in inputs_df.keys():
                input_df = inputs_df[df]
                rows = (input_df.shape[0])
                incorr = []
                for i in range(0, rows-1):
                        var1 = input_df.iloc[i]['Current Year']
                        var2 = input_df.iloc[i]['Prior Year']
                        variance = input_df.iloc[i]['Variance']
                        cal_variance = int(var1)-int(var2)
                        if variance != cal_variance:
                                if roundoff(variance - cal_variance) > 1:
                                        incorr.append(input_df.index[i])
                if len(incorr) != 0:
                        print("Variance values of", ", ".join(incorr), "are incorrect")

                calpbt = input_df.loc['COGS', 'Current Year'] + input_df.loc['OPEX', 'Current Year'] + input_df.loc['Revenue', 'Current Year']
                pbt = input_df.loc['PBT', 'Current Year']
                if roundoff(pbt) != roundoff(calpbt):
                        print(df,": PBT for Current Year is incorrect, please check")
                        print("Calculated PBT is: ", calpbt, " and given PBT is: ", pbt)


                calpbt = input_df.loc['COGS', 'Prior Year'] + input_df.loc['OPEX', 'Prior Year'] + input_df.loc['Revenue', 'Prior Year']
                pbt = input_df.loc['PBT', 'Prior Year']
                if roundoff(pbt) != roundoff(calpbt):
                        print(df,": PBT for Prior Year is incorrect, please check")
                        print("Calculated PBT is: ", calpbt, " and given PBT is: ", pbt, "\n")

Example 1 : PBT for Current Year is incorrect, please check
Calculated PBT is:  43110  and given PBT is:  38110
Example 1 : PBT for Prior Year is incorrect, please check
Calculated PBT is:  21500  and given PBT is:  14500 

Example 2 : PBT for Current Year is incorrect, please check
Calculated PBT is:  42710  and given PBT is:  37710
Example 2 : PBT for Prior Year is incorrect, please check
Calculated PBT is:  21500  and given PBT is:  14500 

Example 3 : PBT for Current Year is incorrect, please check
Calculated PBT is:  -48290  and given PBT is:  -53290
Example 3 : PBT for Prior Year is incorrect, please check
Calculated PBT is:  -28500  and given PBT is:  -35500 

Example 4 : PBT for Current Year is incorrect, please check
Calculated PBT is:  -30400  and given PBT is:  -35400
Example 4 : PBT for Prior Year is incorrect, please check
Calculated PBT is:  -28500  and given PBT is:  -35500 

Example 5 : PBT for Current Year is incorrect, please check
Calculated PBT is:  3600  and given 